### Training base ResNet-20 model (no sparsity) on CIFAR-10

In [102]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision import datasets
from resnet import ResNet20
from datetime import datetime
from tqdm.notebook import tqdm

In [103]:
hyperparameters = {
    'sparsity_type': "base", # nm / entropy / feather / spartan / ses / base (no sparsity)
    'epochs': 200,
    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'batch_size': 128,
}

In [104]:
"""
ResNet-20 archiecture from https://arxiv.org/pdf/1512.03385:

Total of 6n + 2 layers for n=3 gives 20 total layers

1. 3x3 convolutional layer
2. 6n = 18 convolutional layers:
    - 2n —> 6, 16-dim filter layer
    - 2n —> 6, 32-dim filter layer, stride 2
    - 2n —> 6, 64-dim filter layer, stride 2 
3. Last 2 layers:
    - global average pooling
    - 10-way FC
4. softmax


"""

'\nResNet-20 archiecture from https://arxiv.org/pdf/1512.03385:\n\nTotal of 6n + 2 layers for n=3 gives 20 total layers\n\n1. 3x3 convolutional layer\n2. 6n = 18 convolutional layers:\n    - 2n —> 6, 16-dim filter layer\n    - 2n —> 6, 32-dim filter layer, stride 2\n    - 2n —> 6, 64-dim filter layer, stride 2 \n3. Last 2 layers:\n    - global average pooling\n    - 10-way FC\n4. softmax\n\n\n'

In [105]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [106]:
# Define preprocessing transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])


train_dataset = datasets.CIFAR10(root='./outputs/data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./outputs/data', train=False, download=True, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, num_workers=2)

In [107]:
def train(model, train_loader, criterion, optimizer, epoch, log_file):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", ncols=100)
    
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

        pbar.set_postfix(loss=running_loss/(batch_idx+1), accuracy=100.0 * correct / total)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Epoch [{epoch+1}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_loss, accuracy


In [108]:
def test(model, test_loader, criterion, log_file):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    pbar = tqdm(test_loader, desc="Testing", ncols=100)
    
    with torch.no_grad():
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

            pbar.set_postfix(loss=test_loss/(total + inputs.size(0)), accuracy=100.0 * correct / total)

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_test_loss, accuracy

In [109]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

current_learning_rate = 0.1
DECAY = 0.1
DECAY_EPOCHS = [100, 150]
EPOCHS = 200

resnet20_model = ResNet20().to(device)

In [110]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet20_model.parameters(), lr=hyperparameters['lr'], 
                      momentum=hyperparameters['momentum'], weight_decay=hyperparameters['weight_decay'])

In [111]:
sparsity_type = hyperparameters['sparsity_type']
output_base_path = "/home/sg666/Class/ECE661/outputs"
sparsity_folder_path = os.path.join(output_base_path, sparsity_type)

hyperparameter_str = f"epochs_{hyperparameters['epochs']}_lr_{hyperparameters['lr']}_momentum_{hyperparameters['momentum']}_wd_{hyperparameters['weight_decay']}_batch_{hyperparameters['batch_size']}"
output_folder = os.path.join(sparsity_folder_path, hyperparameter_str)

os.makedirs(output_folder, exist_ok=True)

hyperparameter_file = os.path.join(output_folder, 'hyperparameters.txt')
with open(hyperparameter_file, 'w') as f:
    for key, value in hyperparameters.items():
        f.write(f"{key}: {value}\n")

log_file_path = os.path.join(output_folder, 'training_log.txt')

In [112]:
with open(log_file_path, 'w') as log_file:
    log_file.write(f"Training started at {datetime.now()}\n")

    best_accuracy = 0.0

    for epoch in range(hyperparameters['epochs']):
        train_loss, train_accuracy = train(resnet20_model, train_loader, criterion, optimizer, epoch, log_file)
        test_loss, test_accuracy = test(resnet20_model, test_loader, criterion, log_file)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            model_checkpoint_path = os.path.join(output_folder, f"best_model.pth")
            torch.save(resnet20_model.state_dict(), model_checkpoint_path)
            print(f"Saved best model at epoch {epoch+1} with accuracy: {best_accuracy:.2f}%")

        if epoch+1 in DECAY_EPOCHS:
            current_learning_rate = current_learning_rate * DECAY
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            print("Current learning rate has decayed to %f" %current_learning_rate)

    log_file.write(f"Training completed at {datetime.now()}\n")
    log_file.write(f"Best model accuracy: {best_accuracy:.2f}%\n")


Epoch 1:   0%|                                                              | 0/391 [00:00<?, ?it/s]

Testing:   0%|                                                               | 0/79 [00:00<?, ?it/s]

Saved best model at epoch 1 with accuracy: 42.59%


Epoch 2:   0%|                                                              | 0/391 [00:00<?, ?it/s]

Testing:   0%|                                                               | 0/79 [00:00<?, ?it/s]

...

Testing:   0%|                                                               | 0/79 [00:00<?, ?it/s]

Epoch 199:   0%|                                                            | 0/391 [00:00<?, ?it/s]

Testing:   0%|                                                               | 0/79 [00:00<?, ?it/s]

Epoch 200:   0%|                                                            | 0/391 [00:00<?, ?it/s]

Testing:   0%|                                                               | 0/79 [00:00<?, ?it/s]